In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import joblib

In [2]:
from hamlfeeder.data_utils import *

## Deep Learning Model for manage HomeAssistant Alarm based on anothers sensor states

In [3]:
update_database()
df = get_normalized(get_dataframe(get_uuids()))
X = df.loc[:, df.columns != 'target']
y = df.target

Actualizando copia local de la base de datos...


In [4]:
X.head()

,s,mvn,mvns,mvr,mvrs,ls,tv,sun,sl,sps,L,M,X,J,V,S,D
0,0.619499,1,0.627235,0,0.284966,0,1,1,0.543347,0.439747,0,0,1,0,0,0,0
1,0.496415,0,0.003409,1,0.482667,0,1,1,0.549395,0.115405,0,0,0,0,0,1,0
2,0.525594,0,0.520455,0,0.270362,0,0,1,1.000000,0.293414,0,1,0,0,0,0,0
3,0.834978,1,0.813548,1,0.833580,0,1,0,0.105847,0.684812,0,1,0,0,0,0,0
4,0.667794,0,0.697159,1,0.667229,0,0,1,1.000000,0.529426,0,0,0,1,0,0,0


### Features in order: 
* Second of the Day
* Mobile #1 presence
* Mobile #1 last status change since in seconds
* Mobile #2 presence
* Mobile #2 last status change since in seconds
* Lamp #1 status
* Tv #1 status
* Sun (Day/Night)
* Light Sensor #1
* Door last status change since in seconds, WeekDay.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print("Total number of samples: {} and features: {}".format(X.shape[0], X.shape[1]))
print("Train samples: {} Test samples: {}".format(X_train.shape[0], X_test.shape[0]))

Total number of samples: 232 and features: 17
Train samples: 155 Test samples: 77


In [12]:
print(pd.DataFrame({'Percentage': df.groupby(('target')).size() / len(df)}))

        Percentage
target            
0         0.573276
1         0.426724


In [13]:
def create_model():
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(10, activation='relu', kernel_initializer='normal', input_dim=len(X_train.columns)))#Second  Hidden Layer
    classifier.add(Dense(1, activation='relu', kernel_initializer='normal'))
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    return classifier

In [22]:
#model = create_model()
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X_train, y_train, cv=kfold)
print("Results: %.2f%% (+/- %.2f%%)" % (results.mean()*100, results.std()*100))

Results: 87.89% (+/- 8.76%)
